In [5]:
# Exo 1 - Préparation des données 

# importation du fichier
import pandas as pd 
data = pd.read_csv("synthetic.csv")

# Visualisation des données
print(data.head())


      Attr_A     Attr_B       Attr_C      Attr_D      Attr_E       Attr_F  \
0  12.478064  14.784992  1247.541877  100.962061   52.462177  1089.398211   
1   8.264345   8.854181  1389.686814   99.526529   65.106526  1186.523399   
2   9.875571  13.292442   779.077401  123.993772  104.699796  1201.722480   
3   9.207661   9.346913   965.468523   89.176009  102.628284   743.913507   
4   8.863842  12.542969  1096.386230  106.595385  131.813380   883.059615   

      Attr_G     Attr_H     Attr_I      Attr_J       Attr_K     Attr_L  \
0  10.575834   8.375407  10.288159  110.746551   994.367610   9.069350   
1   9.500485  10.088058   9.371983   78.210274   943.089589   9.988919   
2   9.545266  14.266238   9.703551   86.252483  1082.989190  10.084217   
3   9.777953  11.613946   8.912059   96.727873   812.800511   8.621781   
4  10.092974  13.556029  11.649982   21.566576   971.083175  10.072271   

        Attr_M      Attr_N  Class  
0  1027.953917  109.672758      1  
1  1120.317724   83.

In [6]:
# Type de données et valeurs manquantes
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Attr_A  2298 non-null   float64
 1   Attr_B  2298 non-null   float64
 2   Attr_C  2298 non-null   float64
 3   Attr_D  2298 non-null   float64
 4   Attr_E  2298 non-null   float64
 5   Attr_F  2298 non-null   float64
 6   Attr_G  2298 non-null   float64
 7   Attr_H  2298 non-null   float64
 8   Attr_I  2298 non-null   float64
 9   Attr_J  2298 non-null   float64
 10  Attr_K  2298 non-null   float64
 11  Attr_L  2298 non-null   float64
 12  Attr_M  2298 non-null   float64
 13  Attr_N  2298 non-null   float64
 14  Class   2298 non-null   int64  
dtypes: float64(14), int64(1)
memory usage: 269.4 KB
None


In [7]:
# Avoir le nombre d'attributs dans le modèle
print(data.columns)
# 14 attributs dans le modèle

Index(['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G',
       'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N',
       'Class'],
      dtype='object')


In [16]:
# Nombre de classes diff
nbr_class = data['Class'].nunique()
print(nbr_class)
# 4 classes différentes

4


In [ ]:
# combien d'instanes ompte haque classe?
nbr_instances = data['Class'].value_counts()
print(nbr_instances)

In [ ]:
# 